# Setup

In [1]:
!pip install -q --upgrade news-fetch fake_useragent pymilvus

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 80.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.4/222.4 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.9/95.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import sys
import logging

logger = logging.getLogger('stock_recommender')
for handler in logger.handlers[:]:
    logger.removeHandler(handler)

formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

stream_handler = logging.StreamHandler(sys.stdout)
stream_handler.setLevel(logging.DEBUG)
stream_handler.setFormatter(formatter)

logger.addHandler(stream_handler)

In [3]:
# !rm google*
# !wget https://raw.githubusercontent.com/santhoshse7en/news-fetch/refs/heads/master/newsfetch/google.py
# !mv google.py google_local.py

# # issue related to double quote
# ! sed -i "s/\"+\"/'+'/1" google_local.py  # Replacing "+" with '+'

# # Issue related to chrome
# ! sed -i "22i \        options.add_argument('--no-sandbox')" google_local.py

# # making search_url as attribute
# ! sed -i "s/search_url/self.search_url/g" google_local.py  # Replacing "+" with '+'

# # Results
# ! sed -n 15,25p google_local.py

In [4]:
!rm -rf google*
!wget https://raw.githubusercontent.com/pankajr141/experiments/refs/heads/master/Experiments/stock_recommendation/libs/google_search_news_url_extracter.py
!mv google_search_news_url_extracter.py google_local.py

# Import Library from modified code
from google_local import GoogleSearchNewsURLExtractor

--2025-02-21 18:58:11--  https://raw.githubusercontent.com/pankajr141/experiments/refs/heads/master/Experiments/stock_recommendation/libs/google_search_news_url_extracter.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3223 (3.1K) [text/plain]
Saving to: ‘google_search_news_url_extracter.py’

google_search_news_ 100%[===================>]   3.15K  --.-KB/s    in 0s      

2025-02-21 18:58:11 (38.4 MB/s) - ‘google_search_news_url_extracter.py’ saved [3223/3223]



# News - URL through search Engine

In [5]:
def get_stock_news_urls(stock_name, domains, user_agent=None):
    
    urls_news = []
    for domain in domains:
        google = GoogleSearchNewsURLExtractor(keyword=stock_name, news_domain=domain, user_agent=user_agent)
        # print(f"returned_url: {google.returned_url}")

        logger.info(f"domain: {domain}, urls: {len(google.urls)}")
        urls_news.append({
            'domain': domain,
            'stock_name': stock_name,
            'urls': google.urls
        })
    return urls_news

domains = ["https://www.businesstoday.in/", "https://timesofindia.indiatimes.com/", "https://www.thehindu.com"]
stock_name = "tata motors"
user_agent = "Mozilla/5.0 (iPhone; CPU iPhone OS 18_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) GSA/343.0.695551749 Mobile/15E148 Safari/604.1"

url_dict = get_stock_news_urls(stock_name, domains, user_agent)

2025-02-21 18:58:23,784 - INFO - domain: https://www.businesstoday.in/, urls: 24
2025-02-21 18:58:26,520 - INFO - domain: https://timesofindia.indiatimes.com/, urls: 19
2025-02-21 18:58:29,328 - INFO - domain: https://www.thehindu.com, urls: 16


# News - Data per URL

In [6]:
from newsfetch.news import Newspaper
import pandas as pd

def fetch_article_details(url, domain, stock_name):
    news = Newspaper(url=url)

    headline = news.headline
    publish_timestamp = news.date_publish
    article = news.article
    summary = news.summary

    id_suffix = f"{domain}_{stock_name}"

    article_details = {
        'identifier': f"{news.date_publish}_{id_suffix}",
        'timestamp_publish': news.date_publish,
        'headline': news.headline,
        'summary': news.summary,
        'article': news.article,
        'url': url,
        'domain': domain,
        'stock_name': stock_name, 
        'year': pd.to_datetime(news.date_publish).year,
        'month': pd.to_datetime(news.date_publish).month,
        'day': pd.to_datetime(news.date_publish).day
    }
    return article_details

# domain = url_dict[0]['domain']
# stock_name = url_dict[0]['stock_name']
# url = url_dict[0]['urls'][0]
# fetch_article_details(url, domain, stock_name)

In [7]:
from tqdm import tqdm

data = []

success = 0
failure = 0
total_iter = sum([len(x['urls']) for x in url_dict])
with tqdm(total=total_iter) as pbar:
    for url_dict_domain in url_dict:
        domain = url_dict_domain['domain']
        stock_name = url_dict_domain['stock_name']
        urls = url_dict_domain['urls']
        
        for url in urls:
            try:
                article_detail = fetch_article_details(url, domain, stock_name)
                data.append(article_detail)
                success += 1
            except Exception as err:
                # logger.error(f"{err} - {url}")
                failure += 1
                pass
            pbar.update(1)
            pbar.set_description(f"success: {success}, failure: {failure}")

success: 37, failure: 22: 100%|██████████| 59/59 [01:55<00:00,  1.96s/it]


# Vector Database

In [8]:
!cp /kaggle/input/sr-data-gathering-news/stock_recommender.db /kaggle/working

In [9]:
from pymilvus import MilvusClient
client = MilvusClient("./stock_recommender.db")
collection_name = "nr_stock_news"

In [10]:
collections = client.list_collections()
logger.info(f"Availaible collections: {collections}")

if collection_name in collections:
    logger.info(f"collection ({collection_name}) : {client.get_collection_stats(collection_name)}")
    # logger.warning(f"Dropping collection: {collection_name}")
    # client.drop_collection(collection_name)

2025-02-21 19:00:32,139 - INFO - Availaible collections: ['nr_stock_news']
2025-02-21 19:00:32,142 - INFO - collection (nr_stock_news) : {'row_count': 36}


In [11]:
from pymilvus import MilvusClient, DataType

schema = MilvusClient.create_schema(
    auto_id=False,
    enable_dynamic_field=False,
)

schema.add_field(field_name="identifier", datatype=DataType.VARCHAR, max_length=256, is_primary=True)
schema.add_field(field_name="timestamp_publish", datatype=DataType.VARCHAR, max_length=20)
schema.add_field(field_name="headline", datatype=DataType.VARCHAR, max_length=256)
schema.add_field(field_name="summary", datatype=DataType.VARCHAR, max_length=256)
schema.add_field(field_name="article", datatype=DataType.VARCHAR, max_length=2048)
schema.add_field(field_name="url", datatype=DataType.VARCHAR, max_length=256)
schema.add_field(field_name="domain", datatype=DataType.VARCHAR, max_length=30)
schema.add_field(field_name="stock_name", datatype=DataType.VARCHAR, max_length=30)
schema.add_field(field_name="year", datatype=DataType.INT8)
schema.add_field(field_name="month", datatype=DataType.INT8)
schema.add_field(field_name="day", datatype=DataType.INT8)

client.create_collection(
    collection_name=collection_name,
    schema=schema
)

In [12]:
res = client.upsert(
    collection_name=collection_name,
    data=data
)
client.get_collection_stats(collection_name)

{'row_count': 43}

In [13]:
iterator = client.query_iterator(
    collection_name=collection_name,
    batch_size=1,
    output_fields=["identifier"]
)

count=0
while True:
    result = iterator.next()
    
    if not result:
        iterator.close()
        break
    count += 1
    print(count, result)

2025-02-21 19:00:32,381 [WARNING][__setup_ts_by_request]: failed to get mvccTs from milvus server, use client-side ts instead (iterator.py:260)


1 [{'identifier': '2018-07-11 12:07:37_https://www.thehindu.com_tata motors'}]
2 [{'identifier': '2021-07-14 17:53:57_https://www.thehindu.com_tata motors'}]
3 [{'identifier': '2021-08-09 16:29:12_https://timesofindia.indiatimes.com/_tata motors'}]
4 [{'identifier': '2022-06-29 05:27:52_https://www.thehindu.com_tata motors'}]
5 [{'identifier': '2023-06-02 15:36:43_https://www.thehindu.com_tata motors'}]
6 [{'identifier': '2023-10-10 15:43:21_https://www.thehindu.com_tata motors'}]
7 [{'identifier': '2024-02-13 17:12:03_https://www.thehindu.com_tata motors'}]
8 [{'identifier': '2024-03-04 12:52:43_https://www.thehindu.com_tata motors'}]
9 [{'identifier': '2024-04-10 16:13:41_https://www.thehindu.com_tata motors'}]
10 [{'identifier': '2024-06-20 11:59:03_https://www.thehindu.com_tata motors'}]
11 [{'identifier': '2024-08-07 16:58:36_https://www.thehindu.com_tata motors'}]
12 [{'identifier': '2024-10-01 20:42:00_https://www.thehindu.com_tata motors'}]
13 [{'identifier': '2024-10-10 08:43: